In [28]:
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Dataset= pd.read_csv("Training.csv")


Dataset=Dataset.dropna() # remove null values
Dataset=Dataset.drop("Id",axis=1) # remove sample information
#print(Dataset.value_counts(Dataset["Class"]))
Labels=Dataset[["Class"]].copy() # generate separate class dataframe
Dataset=Dataset.drop("Class",axis=1) # drop class from feature dataframe
mapping = {'A': 1, 'B': 0} # change object from str to int 
Dataset['EJ'] = Dataset['EJ'].replace(mapping)
#print(Dataset.shape)
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)
#print(type(Dataset_train))
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)
#print(AltLabels.value_counts(AltLabels["Class"]))
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels=tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)
print(len(Labels_train))
DiseaseModel=keras.Sequential([layers.Dense(300,activation="relu"),layers.Dense(1,activation="sigmoid")])
DiseaseModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
DiseaseModel.fit(AltDataset,AltLabels,epochs=30,batch_size=150)
print("I will determine the accuracy of my test data.")
Evaluations=DiseaseModel.evaluate(Dataset_test,Labels_test)

274
Epoch 1/30
3/3 [==============================] - 1s 4ms/step - loss: 1037.3093 - accuracy: 0.5023
Epoch 2/30
3/3 [==============================] - 0s 5ms/step - loss: 350.4420 - accuracy: 0.4480
Epoch 3/30
3/3 [==============================] - 0s 8ms/step - loss: 329.4302 - accuracy: 0.5136
Epoch 4/30
3/3 [==============================] - 0s 7ms/step - loss: 236.4007 - accuracy: 0.5837
Epoch 5/30
3/3 [==============================] - 0s 3ms/step - loss: 191.3472 - accuracy: 0.5633
Epoch 6/30
3/3 [==============================] - 0s 8ms/step - loss: 193.3173 - accuracy: 0.5860
Epoch 7/30
3/3 [==============================] - 0s 20ms/step - loss: 174.9352 - accuracy: 0.5814
Epoch 8/30
3/3 [==============================] - 0s 5ms/step - loss: 108.3465 - accuracy: 0.6109
Epoch 9/30
3/3 [==============================] - 0s 4ms/step - loss: 60.8801 - accuracy: 0.6041
Epoch 10/30
3/3 [==============================] - 0s 5ms/step - loss: 53.3685 - accuracy: 0.5882
Epoch 11/30
3/3

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def create_model():
    model = Sequential([Dense(140, activation="relu"),Dense(30, activation="swish"), Dense(1, activation="sigmoid")])
    return model

loss_function = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

population_size=100


population=[[create_model() for amount in range(2)] for size in range(population_size)]

optimizer = tf.keras.optimizers.legacy.Adam()


for generation in range(min(10, len(AltDataset))):  # For each generation
    losses = []
    for Individual in population:
        Individual_Loss = 0
        sample, label = AltDataset[generation], AltLabels[generation]
        for model in Individual:
            with tf.GradientTape() as tape:
                prediction = model(sample[None, ...], training=True)
                loss = loss_function(label[None, ...], prediction)

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Use the same optimizer instance

            Individual_Loss += loss    
        losses.append(Individual_Loss/2)


    top_individuals_indices= (np.argsort(losses))[0:int((len(population)/2)+1)]
    #The following line identifies the positions of the individuals (models) 
    # with the lowest losses in the population. Specifically, it selects just over 
    # half of the population, ranking them by their loss values. This subset of 
    # indices can be used to select the top-performing individuals for further 
    # genetic operations such as crossover and mutation, thereby ensuring that 
    # the traits of the best-performing models are carried over to the next generation.
    
    
    def generate_haploid_cluster(individual, cluster_size): 
        # This function takes an individual organism containing two models (analogous to paired chromosomes) 
        # and generates a cluster of gametes (like sperm), each gamete is a single model which is a combination of the two models (analogous to process of meiosis).

        haploid_cluster = [] # List to store the generated gametes

        for gamete_index in range(cluster_size): # Loop to create the desired number of gametes
            paired_chromosomes = individual # Retrieve the two models representing the individual's paired chromosomes
            gamete = create_model() # Create a new model with the same architecture
            gamete.build((None, sample.shape[0])) # Initialize the weights of the gamete model based on the anticipated input size

            # Loop through the weight tensors in the gamete model
            for weight_index in range(len(gamete.weights)): 
                # Create a mask of random 0s and 1s to determine which genetic material to inherit from each paired chromosome
                mask = np.random.choice([0, 1], size=gamete.weights[weight_index].shape.as_list())
                
                # Use the mask to combine genetic material from both paired chromosomes, element by element
                combined_weights = np.where(mask, paired_chromosomes[0].weights[weight_index].numpy(), paired_chromosomes[1].weights[weight_index].numpy())

                # Assign the combined genetic material to the gamete model
                gamete.weights[weight_index].assign(combined_weights)

            haploid_cluster.append(gamete) # Add the gamete to the cluster

        return haploid_cluster # Return the cluster of gametes
    

    def gamete_selection(haploid_cluster, generation, num_gametes=4):
        sample, label = AltDataset[0:generation], AltLabels[0:generation]

        gametes_fitness_scores = []
        for gamete in haploid_cluster:
            with tf.GradientTape() as tape:
                prediction = gamete(sample, training=True)
                loss_fitness_score = loss_function(label, prediction)
            gametes_fitness_scores.append((gamete, loss_fitness_score))
            
        # Sort the gametes by their fitness scores and select the top N
        sorted_gametes = sorted(gametes_fitness_scores, key=lambda x: x[1])[:num_gametes]
        return [gamete[0] for gamete in sorted_gametes]


    def offspring_fertilisation(parent_one_gamete,parent_two_gamete): #Two gametes come together to form a whole individual with a complete set of chromosomes (two models)
        offspring=[parent_one_gamete,parent_two_gamete]
        return(offspring)

       # Create the next generation
    count = 0

    next_generation = []  # Make sure to initialize the next_generation list
    for i in range(len(top_individuals_indices) - 1):  # Pairing individuals
        # Only proceed with even counts
        if count % 2 != 0:
            count += 1
            continue

        # Generate haploid clusters for parent individuals
        haploid_cluster_parent_one = generate_haploid_cluster(population[top_individuals_indices[count]], cluster_size=10)
        haploid_cluster_parent_two = generate_haploid_cluster(population[top_individuals_indices[count + 1]], cluster_size=10)

        # Select the best gametes from each cluster
        parent_one_gamete = gamete_selection(haploid_cluster_parent_one, generation)
        parent_two_gamete = gamete_selection(haploid_cluster_parent_two, generation)

        # Fertilize the selected gametes to create offspring
        offspring1 = offspring_fertilisation(parent_one_gamete[0], parent_two_gamete[0])
        offspring2 = offspring_fertilisation(parent_one_gamete[1], parent_two_gamete[1])
        offspring3 = offspring_fertilisation(parent_one_gamete[2],parent_two_gamete[2])
        offspring4 = offspring_fertilisation(parent_one_gamete[3],parent_two_gamete[3])
        # Add the offspring to the next generation
        next_generation.append(offspring1)
        next_generation.append(offspring2)
        next_generation.append(offspring3)
        next_generation.append(offspring4)
        count += 1
    population=next_generation
    print(population)


    


[[<keras.engine.sequential.Sequential object at 0x0000013F530D8670>, <keras.engine.sequential.Sequential object at 0x0000013F52EFCDF0>], [<keras.engine.sequential.Sequential object at 0x0000013F530B0700>, <keras.engine.sequential.Sequential object at 0x0000013F52EDC3A0>], [<keras.engine.sequential.Sequential object at 0x0000013F530915B0>, <keras.engine.sequential.Sequential object at 0x0000013F52ECAE80>], [<keras.engine.sequential.Sequential object at 0x0000013F5307E370>, <keras.engine.sequential.Sequential object at 0x0000013F52E9FE20>], [<keras.engine.sequential.Sequential object at 0x0000013F59DAE0A0>, <keras.engine.sequential.Sequential object at 0x0000013F52FBA640>], [<keras.engine.sequential.Sequential object at 0x0000013F51E11790>, <keras.engine.sequential.Sequential object at 0x0000013F51CC9A90>], [<keras.engine.sequential.Sequential object at 0x0000013F51DD0F40>, <keras.engine.sequential.Sequential object at 0x0000013F51CA7F70>], [<keras.engine.sequential.Sequential object at 

In [33]:
print(population)
from tensorflow.keras.metrics import BinaryAccuracy

for i, individual in enumerate(population):  
      specificity=0
      sensitivity=0
      precision_class_one=0
      Negative_Predictive_Value=0
      accuracy=0
      for model in individual:
            y_pred = model.predict(Dataset_test)  # Assuming Dataset_test is your test data
            y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels (0 or 1)

            conf_matrix = np.zeros((2, 2), dtype=int)
            conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
            conf_matrix[0, 1] = np.sum((Labels_test.values == 0) & (y_pred == 1))  # False Positives (FP)
            conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
            conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

            # Extract true negatives (TN), false positives (FP), true positives (TP), and false negatives (FN) from the confusion matrix
            TN, FP, TP, FN = conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 1], conf_matrix[1, 0]

            # Calculate specificity, sensitivity, precision, NPV, and accuracy
            specificity = specificity+(TN / (TN + FP))
            sensitivity = sensitivity+(TP / (TP + FN))
            precision_class_one = precision_class_one+(TP / (TP + FP))
            Negative_Predictive_Value =Negative_Predictive_Value+( TN / (TN + FN))
            accuracy =accuracy+((TP + TN) / (TP + TN + FP + FN))

            
      # Print the metrics
      #"""
      print(f"Individual {i+1}")
      print(f"Sensitivity: {sensitivity/2 * 100:.2f}%,",
            f"Specificity: {specificity/2 * 100:.2f}%,",
            f"Accuracy: {accuracy/2 * 100:.2f}%,",
            f"Negative Predictive Value: {Negative_Predictive_Value/2 * 100:.2f}%,",
            f"Precision (Class One): {precision_class_one/2 * 100:.2f}%")
      print(f"True Negatives (TN): {TN},",
            f"False Positives (FP): {FP},",
            f"True Positives (TP): {TP},",
            f"False Negatives (FN): {FN}")
      #"""


Proportion = Labels_test["Class"].value_counts()
print("By random chance the precision would be {:.2f}%".format(Proportion[1] / (Proportion[0] + Proportion[1]) * 100))


[[<keras.engine.sequential.Sequential object at 0x0000013EAA3A2760>, <keras.engine.sequential.Sequential object at 0x0000013F5C2F4AF0>], [<keras.engine.sequential.Sequential object at 0x0000013F56C7AF10>, <keras.engine.sequential.Sequential object at 0x0000013F546502B0>], [<keras.engine.sequential.Sequential object at 0x0000013EECC31F70>, <keras.engine.sequential.Sequential object at 0x0000013F58651C70>], [<keras.engine.sequential.Sequential object at 0x0000013EB722A7C0>, <keras.engine.sequential.Sequential object at 0x0000013F557AE160>], [<keras.engine.sequential.Sequential object at 0x0000013F4EE3DDC0>, <keras.engine.sequential.Sequential object at 0x0000013EF1344700>], [<keras.engine.sequential.Sequential object at 0x0000013F4EC55430>, <keras.engine.sequential.Sequential object at 0x0000013F4C1535B0>], [<keras.engine.sequential.Sequential object at 0x0000013F4EC4C610>, <keras.engine.sequential.Sequential object at 0x0000013F5313FEB0>], [<keras.engine.sequential.Sequential object at 

C:\Users\James\AppData\Local\Temp\ipykernel_10440\3732804848.py:26: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = precision_class_one+(TP / (TP + FP))


9/9 [==============================] - 0s 2ms/step
Individual 13
Sensitivity: 21.43%, Specificity: 93.78%, Accuracy: 80.84%, Negative Predictive Value: 84.76%, Precision (Class One): 30.95%
True Negatives (TN): 203, False Positives (FP): 22, True Positives (TP): 20, False Negatives (FN): 29
9/9 [==============================] - 0s 2ms/step
Individual 14
Sensitivity: 12.24%, Specificity: 76.44%, Accuracy: 64.96%, Negative Predictive Value: 79.28%, Precision (Class One): 14.87%
True Negatives (TN): 123, False Positives (FP): 102, True Positives (TP): 11, False Negatives (FN): 38
9/9 [==============================] - 0s 2ms/step
Individual 15
Sensitivity: 63.27%, Specificity: 46.44%, Accuracy: 49.45%, Negative Predictive Value: 92.62%, Precision (Class One): 30.64%
True Negatives (TN): 1, False Positives (FP): 224, True Positives (TP): 49, False Negatives (FN): 0
1/9 [==>...........................] - ETA: 0s

C:\Users\James\AppData\Local\Temp\ipykernel_10440\3732804848.py:26: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = precision_class_one+(TP / (TP + FP))


9/9 [==============================] - 0s 2ms/step
Individual 16
Sensitivity: 29.59%, Specificity: 72.22%, Accuracy: 64.60%, Negative Predictive Value: 82.73%, Precision (Class One): nan%
True Negatives (TN): 100, False Positives (FP): 125, True Positives (TP): 29, False Negatives (FN): 20
9/9 [==============================] - 0s 2ms/step
Individual 17
Sensitivity: 29.59%, Specificity: 59.56%, Accuracy: 54.20%, Negative Predictive Value: 75.40%, Precision (Class One): 6.90%
True Negatives (TN): 44, False Positives (FP): 181, True Positives (TP): 29, False Negatives (FN): 20
9/9 [==============================] - 0s 2ms/step
Individual 18
Sensitivity: 42.86%, Specificity: 66.89%, Accuracy: 62.59%, Negative Predictive Value: 86.84%, Precision (Class One): nan%
True Negatives (TN): 76, False Positives (FP): 149, True Positives (TP): 42, False Negatives (FN): 7


...

9/9 [==============================] - 0s 1ms/step
Individual 77
Sensitivity: 10.20%, Specificity: 95.56%, Accuracy: 80.29%, Negative Predictive Value: 83.03%, Precision (Class One): 30.00%
True Negatives (TN): 217, False Positives (FP): 8, True Positives (TP): 2, False Negatives (FN): 47
9/9 [==============================] - 0s 1ms/step
Individual 78
Sensitivity: 23.47%, Specificity: 88.44%, Accuracy: 76.82%, Negative Predictive Value: 84.51%, Precision (Class One): 17.16%
True Negatives (TN): 181, False Positives (FP): 44, True Positives (TP): 23, False Negatives (FN): 26
9/9 [==============================] - 0s 1ms/step
Individual 79
Sensitivity: 11.22%, Specificity: 82.89%, Accuracy: 70.07%, Negative Predictive Value: 81.09%, Precision (Class One): 12.27%
True Negatives (TN): 190, False Positives (FP): 35, True Positives (TP): 4, False Negatives (FN): 45
9/9 [==============================] - 0s 2ms/step
Individual 80
Sensitivity: 18.37%, Specificity: 66.89%, Accuracy: 58.21%, Ne

C:\Users\James\AppData\Local\Temp\ipykernel_10440\3732804848.py:26: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = precision_class_one+(TP / (TP + FP))


9/9 [==============================] - 0s 5ms/step
Individual 89
Sensitivity: 24.49%, Specificity: 65.33%, Accuracy: 58.03%, Negative Predictive Value: 78.19%, Precision (Class One): 18.78%
True Negatives (TN): 208, False Positives (FP): 17, True Positives (TP): 6, False Negatives (FN): 43
9/9 [==============================] - 0s 2ms/step
Individual 90
Sensitivity: 2.04%, Specificity: 99.11%, Accuracy: 81.75%, Negative Predictive Value: 82.29%, Precision (Class One): 37.50%
True Negatives (TN): 222, False Positives (FP): 3, True Positives (TP): 1, False Negatives (FN): 48
9/9 [==============================] - 0s 2ms/step
Individual 91
Sensitivity: 7.14%, Specificity: 94.67%, Accuracy: 79.01%, Negative Predictive Value: 82.42%, Precision (Class One): 11.67%
True Negatives (TN): 202, False Positives (FP): 23, True Positives (TP): 7, False Negatives (FN): 42
9/9 [==============================] - 0s 2ms/step
Individual 92
Sensitivity: 11.22%, Specificity: 70.00%, Accuracy: 59.49%, Negat

C:\Users\James\AppData\Local\Temp\ipykernel_10440\3732804848.py:26: RuntimeWarning: invalid value encountered in long_scalars
  precision_class_one = precision_class_one+(TP / (TP + FP))


9/9 [==============================] - 0s 2ms/step
Individual 99
Sensitivity: 16.33%, Specificity: 54.22%, Accuracy: 47.45%, Negative Predictive Value: 74.85%, Precision (Class One): 7.21%
True Negatives (TN): 120, False Positives (FP): 105, True Positives (TP): 6, False Negatives (FN): 43
9/9 [==============================] - 0s 2ms/step
Individual 100
Sensitivity: 46.94%, Specificity: 44.44%, Accuracy: 44.89%, Negative Predictive Value: 75.41%, Precision (Class One): 17.33%
True Negatives (TN): 154, False Positives (FP): 71, True Positives (TP): 20, False Negatives (FN): 29
By random chance the precision would be 17.88%


In [34]:
import numpy as np

# Assuming y_true and y_pred are numpy arrays representing the true labels and predicted labels, respectively.
# 1 represents the positive class, and 0 represents the negative class.
y_pred=model.predict(Dataset_test)
# Compute the confusion matrix
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(f"Specificity: {specificity * 100:.2f}%",
      f"Sensitivity: {sensitivity * 100:.2f}%",
      f"Accuracy: {accuracy * 100:.2f}%",
      f"Negative Predictive Value: {Negative_Predictive_Value * 100:.2f}%",
      f"Precision (Class One): {precision_class_one * 100:.2f}%")
print(f"True Negatives (TN): {TN}",
      f"False Positives (FP): {FP}",
      f"True Positives (TP): {TP}",
      f"False Negatives (FN): {FN}")


9/9 [==============================] - 0s 2ms/step
Specificity: 68.42% Sensitivity: 41.67% Accuracy: 63.42% Negative Predictive Value: 83.63% Precision (Class One): 23.26%
True Negatives (TN): 143 False Positives (FP): 66 True Positives (TP): 20 False Negatives (FN): 28


In [36]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
y_pred = DiseaseModel.predict(Dataset_test)  # assuming x_test is your test data
y_pred = (y_pred > 0.5).astype(int)
conf_matrix = np.zeros((2, 2), dtype=int)
conf_matrix[0, 0] = np.sum((Labels_test.values == 0) & (y_pred == 0))  # True Negatives (TN)
conf_matrix[0, 1] = np.sum((Labels_test.values== 0) & (y_pred == 1))  # False Positives (FP)
conf_matrix[1, 0] = np.sum((Labels_test.values == 1) & (y_pred == 0))  # False Negatives (FN)
conf_matrix[1, 1] = np.sum((Labels_test.values == 1) & (y_pred == 1))  # True Positives (TP)

# Extract true negatives (TN) and false positives (FP) from the confusion matrix
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
TP = conf_matrix[1, 1]
FN = conf_matrix[1, 0]
# Calculate specificity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)
precision_class_one=TP/(TP+FP)
Negative_Predictive_Value=TN/(TN+FN)
accuracy=(TP+TN)/(TP+TN+FP+FN)
print(sensitivity*100,specificity*100,accuracy*100,Negative_Predictive_Value*100,precision_class_one*100)
print(TN,FP,TP,FN)
#Sensitivity: 44.90%, Specificity: 82.22%, Accuracy: 75.55%, Negative Predictive Value: 87.26%, Precision (Class One): 35.48%


9/9 [==============================] - 0s 2ms/step
69.38775510204081 79.55555555555556 77.73722627737226 92.26804123711341 42.5
179 46 34 15
